In [1]:
import sys
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from train import twoHiddenNet
from utils.img_to_vec import Img2Vec

import os

In [2]:
MODEL_PATH = "../model/age_est_model.pt"

In [3]:
def get_feats(image_path):
    fe = Img2Vec(cuda=False)  # change this if you use Cuda version of the PyTorch.
    img = Image.open(image_path)
    img = img.resize((224, 224))
    feats = fe.get_vec(img).reshape(1, -1)
    return feats

In [4]:
def get_model():
    model = twoHiddenNet()
    model.load_state_dict(torch.load(MODEL_PATH))
    return model

In [5]:
def estimate_age_from_image(image_path):

    image_feats = get_feats(image_path)

    model = get_model()
    estimated_age = model(Variable(torch.from_numpy(image_feats).float()))
    return estimated_age.data.cpu().numpy()[0][0]

In [ ]:
def test(test_data_path):
    folders = os.listdir(test_data_path)
    for folder in folders:
        

In [ ]:
'/home/dongwenjian/Jupyter/DeepLearningWithPyTorch/projet/data5'